# Fuzzy Matching
Source: https://www.datacamp.com/community/tutorials/fuzzy-string-python

# Levenshtein Distance 

In [10]:
import Levenshtein as lev

In [19]:
Str1 = "Apple Inc."
Str2 = "apple Inc"

Distance = lev.distance(Str1,Str2)
print(f"We need {Distance} edits to convert [{Str1}] into [{Str2}]")

Ratio = lev.ratio(Str1.lower(),Str2.lower())
print(f"The Lev Ratio is {round(Ratio, 2)}")

We need 2 edits to convert [Apple Inc.] into [apple Inc]
The Lev Ratio is 0.95


# FuzzyWuzzy Package

In [21]:
from fuzzywuzzy import fuzz

- The FuzzyWuzzy package allows both ratio and partial ratio functions to act as metrics of similarities between two strings.
- Note that Partial Ratio permits additional words in the first string but these words remain irrelevant.

In [27]:
Str1 = "Los Angeles Lakers"
Str2 = "Lakers"
Str3 = "Los Angeles lakers"
Ratio = fuzz.ratio(Str1,Str2)
Partial_Ratio12 = fuzz.partial_ratio(Str1,Str2)
Partial_Ratio32 = fuzz.partial_ratio(Str3,Str2)
print(Ratio)
print(Partial_Ratio12)
print(Partial_Ratio32)

50
100
83


# fuzz.token_sort
- The FuzzyWuzzy package has another useful function: fuzz.token 
- This is useful when two strings represent the same object, but differ in order. eg. "A and B" === "B and A"
- This function deals : 
1. tokenize the strings
2. preprocess them by turning them to lower case and getting rid of punctuation. 
3. tokens get sorted alphabetically  
4. After that, a simple fuzz.ratio() is applied to obtain the similarity percentage. 

# fuzz.token_set
Additional step added:

5. token_set_ratio transform each string into a set and performs an intersection operation. This removes Irrelevant words that only appear in one set of words 

In [3]:
Str1 = "The supreme court case of Nixon vs The United States"
Str2 = "Nixon v. United States"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
Token_Set_Ratio = fuzz.token_set_ratio(Str1,Str2)
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)
print(Token_Set_Ratio)

57
77
58
95


# fuzz.process
- the fuzzywuzzy package has a module called process 
- That calculates the string with the highest similarity out of a vector of strings.

In [4]:
from fuzzywuzzy import process
str2Match = "apple inc"
strOptions = ["Apple Inc.","apple park","apple incorporated","iphone"]
Ratios = process.extract(str2Match,strOptions)
print(Ratios)
# You can also select the string with the highest matching percentage
highest = process.extractOne(str2Match,strOptions)
print(highest)

[('Apple Inc.', 100), ('apple incorporated', 90), ('apple park', 67), ('iphone', 30)]
('Apple Inc.', 100)


# Hard coded Levenshtein Distance

In [1]:
import numpy as np
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

In [6]:
Str1 = "Apple Inc."
Str2 = "apple Inc"

In [8]:
Distance = levenshtein_ratio_and_distance(Str1,Str2)
print(Distance) 
# Edits needed for conversion: 
# 1. A Uppercase to a Lowercase   
# 2. Delete full stop.

The strings are 2 edits away


In [4]:
Ratio = levenshtein_ratio_and_distance(Str1,Str2,ratio_calc = True)
print(Ratio)

0.8421052631578947
